# Lingyi's Course IX Capstone Week3 Section2

# Section 1

## 1.1

#### -Download and import neccessary packages and functions- 

In [1]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import DataFrame
import numpy as np
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1.2

#### -Retrieve and organize data-

In [6]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [7]:
i=0
store = []
for data in soup.find_all('td'):
    i = i+1
    store = store + [data.text]
a = 0
pc = []
for a in range(0,573,3):
    pc = pc + [store[a]]    
b = 1
borough = []
for b in range(1,573,3):
    borough = borough + [store[b]]
c = 2
neighborhood = []
for c in range(2,573,3):
    neighborhood = neighborhood + [store[c]]    

In [8]:
column1 = pd.DataFrame(pc,columns=['PostalCode'])
column2 = pd.DataFrame(borough, columns = ['Borough'])
column3 = pd.DataFrame(neighborhood, columns = ['Neighborhood'])
column1 = column1.drop(column1.index[180:191])
column2 = column2.drop(column2.index[180:191])
column3 = column3.drop(column3.index[180:191])
column1['PostalCode'] = column1['PostalCode'].replace({'\n':''},regex= True)
column2['Borough'] = column2['Borough'].replace({'\n':''}, regex = True)
column3['Neighborhood'] = column3['Neighborhood'].replace({'\n':''},regex= True)
column3['Neighborhood'] = column3['Neighborhood'].replace({'/':','}, regex = True)
df = pd.concat([column1, column2, column3], axis=1)
df['Borough'].replace("Not assigned", np.nan, inplace = True)
df.dropna(subset = ["Borough"], inplace=True)

# Section2

## 2.1

#### -Retrieve Geospatial_data CSV-

In [9]:
file = 'http://cocl.us/Geospatial_data'
geo = pd.read_csv(file)

#### -Merge Toronot datas and Geospatial coordinates Dataframes into One Dataframe-

In [10]:
dft = pd.merge(left = df, right = geo, left_on = 'PostalCode', right_on = 'Postal Code')
#Drop duplicate column 'Postal Code'
dft = dft.drop(['Postal Code'], axis=1)

In [11]:
dft.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937


## 2.2 Visualize Map

#### -Use geopy library to get the latitude and longitude values of Toronto-


In [12]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### -Create a map of Toronto with neighborhoods superimposed on top-


In [13]:
#simplified and create dataframe of Toronto
toronto = dft[dft['Borough'].str.contains('Toronto')]
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [14]:
# create map of Toronto using latitude and longitude values
mapt = folium.Map(location =[latitude, longitude],zoom_start =10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dft['Latitude'],dft['Longitude'],dft['Borough'],dft['Neighborhood']):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#31b5cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(mapt)
mapt

## 2.3 Explore Neighborhood in Toronto

#### -Define foursquare credentials and version-

In [15]:
CLIENT_ID = 'CFU5JPRQFUHRQNUTSOE4LSCP2MGFSXOAY4QTTUGARQV1VI3W'
CLIENT_SECRET = 'R3BQC1SMFA1XHLDKPDJ5MWID0BDTH5TQXJVVLOHW2KVML1YO' 
VERSION = '20200506'

#### _Define a function to explore all the neighborhoods in Toronto using foursquare-

In [16]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
                # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']   
    return(nearby_venues)

#### -Run the above function on each neighborhood and create a new dataframe called toronto_venues-

In [17]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District , Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Rosed

In [18]:
print(toronto_venues.shape)
toronto_venues.groupby('Neighborhood').count()

(1618, 7)


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing Centre,15,15,15,15,15,15
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",15,15,15,15,15,15
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,78,78,78,78,78,78
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


#### -Amount of unique categories curated from all the returned venues-

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


## 2.4 Analyze each Neighborhood

In [20]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#new dataframe size
toronto_onehot.shape

(1618, 232)

#### -Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category-

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.015873,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [23]:
#new size
toronto_grouped.shape

(39, 232)

#### -Top 5 most common venues of each neighborhood-

In [24]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3          Restaurant  0.03
4            Beer Bar  0.03


----Brockton , Parkdale Village , Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4   Burrito Place  0.05


----Business reply mail Processing Centre----
                  venue  freq
0           Pizza Place  0.07
1         Auto Workshop  0.07
2            Smoke Shop  0.07
3  Fast Food Restaurant  0.07
4                   Spa  0.07


----CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport----
             venue  freq
0   Airport Lounge  0.13
1  Airport Service  0.13
2            Plane  0.07
3          Airport  0.07
4         Boutique  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.

#### Put above datas into a Pandas dataframe

#### -sort venues in descending order-

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### -create new dataframe and display top 10 venues in each neighborhood-

In [26]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Restaurant,Beer Bar,Portuguese Restaurant,Butcher
1,"Brockton , Parkdale Village , Exhibition Place",Café,Breakfast Spot,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Stadium,Intersection,Bar
2,Business reply mail Processing Centre,Light Rail Station,Garden,Restaurant,Auto Workshop,Fast Food Restaurant,Farmers Market,Spa,Brewery,Smoke Shop,Garden Center
3,"CN Tower , King and Spadina , Railway Lands , ...",Airport Lounge,Airport Service,Coffee Shop,Harbor / Marina,Plane,Rental Car Location,Sculpture Garden,Boutique,Boat or Ferry,Airport Terminal
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Ice Cream Shop,Middle Eastern Restaurant,Burger Joint,Thai Restaurant,Salad Place


## 2.5 Cluster Neighborhoods

#### -Run K-means to cluster the neighborhoods into 4 clusters-

In [27]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

#### -Create new dataframe to include clusters as well as top 10 venues of each neighborhood-

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Beer Bar,Sandwich Place,Burrito Place,Café,Park,Creperie
9,M5B,Downtown Toronto,"Garden District , Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Theater
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Cosmetics Shop,Creperie,Lingerie Store,Department Store
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### -Visualizing resulting clusters-

In [29]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 2.6 Examine Clusters

#### Cluster 1

#### -Key words: Park, Gym, Trail, Playground-

Boroughs in this cluster contains park, trail, playground and etc. as theirs most common venues. These three boroughs will be a good fit for people who enjoy outdoor exercises and quite neighborhoods to live in.

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,0,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
83,Central Toronto,0,Trail,Gym,Playground,Tennis Court,Distribution Center,Department Store,Dessert Shop,Diner,Discount Store,Women's Store
91,Downtown Toronto,0,Park,Playground,Trail,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


#### Cluster 2

#### -Key words: Coffee shop, Clothing store, Cocktail Bar, Sushi Restaurant, Pub, Bakery-

Boroughs in this cluster contain mostly coffee shop, clothing shop, health food store as theirs most common venues. For people who enjoy hanging out with friends and do cook a lot, those boroughs will be a good fit for them to live in.

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Pub,Bakery,Park,Breakfast Spot,Café,Theater,Yoga Studio,Mexican Restaurant,Shoe Store
4,Downtown Toronto,1,Coffee Shop,Sushi Restaurant,Diner,Yoga Studio,Beer Bar,Sandwich Place,Burrito Place,Café,Park,Creperie
9,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Bubble Tea Shop,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Italian Restaurant,Theater
15,Downtown Toronto,1,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Cosmetics Shop,Creperie,Lingerie Store,Department Store
19,East Toronto,1,Health Food Store,Trail,Pub,Women's Store,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant
20,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Bakery,Seafood Restaurant,Restaurant,Beer Bar,Portuguese Restaurant,Butcher
24,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Ice Cream Shop,Middle Eastern Restaurant,Burger Joint,Thai Restaurant,Salad Place
25,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Athletics & Sports,Candy Store,Diner,Italian Restaurant,Baby Store,Nightclub
30,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Deli / Bodega,Clothing Store,Gym,Thai Restaurant,Hotel,Concert Hall,Bakery
31,West Toronto,1,Bakery,Pharmacy,Music Venue,Supermarket,Middle Eastern Restaurant,Bar,Recording Studio,Café,Pool,Bank


#### Cluster 3

#### -Key words: Garden, Women's Store-

The borough in this cluster contains garden, women's store and etc. as its most common venues. For people who enjoy gardening and prefer to live close to the garden, this will be a good fit for them.

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,2,Garden,Women's Store,Deli / Bodega,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4

#### -Key words: Bus Line, Park, Swim School-

The borough in this cluster contains bus line, park and swim school as its most common venues. I believe this area will be a good fit for people who do not want to live far away from public transportation as well as enjoy exercising every day.

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,3,Bus Line,Park,Swim School,Women's Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


***

## Thank you for watching!! Have a great day!! :D